# Create an interface between two materials with no strain matching

Interface created from two slabs with provided terminations. The interface is created without strain matching, so the film is wrapped into the substrate slab cell without any scaling of atomic coordinates.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials
2. Set notebook settings
2. Create slab configurations, view possible terminations and select the termination pair for the interface
3. Create the interface with no strain matching based on the selected confgiurations
4. Visualize the interface
5. Pass the interface to the outside runtime


## 1. Prepare the Environment
### 1.1. Set up the notebook 

Set the following flags to control the notebook behavior:

In [ ]:
IS_TERMINATIONS_SELECTION_INTERACTIVE = False # Enable interactive selection of terminations via UI prompt
ENABLE_FILM_SCALING = False # Enable scaling of the film slab to match the substrate lattice


### 1.2. Install the Required Packages
Note: the step executes only when using Pyodide. For other environments (e.g. Colab, JupyterLab), the packages should be installed via `pip install .` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb", "../../config.yml")

### 1.3. Get input materials and assign `substrate` and `film`


In [ ]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))
substrate = materials[0]
film = materials[1]

### 1.4. Visualize Substrate and Film

In [ ]:
from utils.visualize import visualize_materials as visualize
visualize([substrate, film], repetitions=[3, 3, 1], rotation="0x")

## 2. Create Slabs for the Interface

### 2.1. Create Substrate and Layer Slabs via SlabGenerator
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.
Define the substrate slab cell that will be used as a base for the interface and the film slab cell that will be placed on top of the substrate slab.

In [ ]:
from mat3ra.made.tools.build.slab import SlabConfiguration

substrate_slab_configuration = SlabConfiguration(
    bulk=substrate,
    # the Miller indices of the interfacial plane of the substrate
    miller_indices=(1,1,1),
    thickness=3,
    vacuum=3,
    xy_supercell_matrix=[[1, 0], [0, 1]],
    use_orthogonal_z=True
)

film_slab_configuration = SlabConfiguration(
    bulk=film,
    # the Miller indices of the interfacial plane of the layer
    miller_indices=(0, 0, 1),
    thickness=1,
    vacuum=1,
    xy_supercell_matrix=[[1, 0], [0, 1]],
    use_orthogonal_z=True
)

substrate_slab_terminations = substrate_slab_configuration.terminations
film_slab_terminations = film_slab_configuration.terminations

print(f"Substrate terminations: {substrate_slab_terminations}")
print(f"Film terminations: {film_slab_terminations}")

### 2.2. Visualize slabs for possible terminations

In [ ]:
for termination in substrate_slab_terminations:
    visualize(substrate_slab_configuration.get_material(termination=termination), repetitions=[3, 3, 1], rotation="-90x", title=f"Termination: {termination}")
    
for termination in film_slab_terminations:
    visualize(film_slab_configuration.get_material(termination=termination), repetitions=[3, 3, 1], rotation="-90x", title=f"Termination: {termination}")
    

### 2.3. Select the termination pair for the interface
Select termination that will be facing interfacial plane for substrate and film

In [ ]:
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide

substrate_termination = substrate_slab_terminations[0]
film_termination = film_slab_terminations[0]

if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        substrate_termination = await ui_prompt_select_array_element_by_index_pyodide(substrate_slab_terminations, element_name="substrate termination")
        film_termination = await ui_prompt_select_array_element_by_index_pyodide(film_slab_terminations, element_name="film termination")
    else:
        substrate_termination = ui_prompt_select_array_element_by_index(substrate_slab_terminations, element_name="substrate termination")
        film_termination = ui_prompt_select_array_element_by_index(film_slab_terminations, element_name="film termination")

termination_pair = (substrate_termination, film_termination)

## 3. Create an interface
### 3.1. Initialize the Interface Builder
We use no strain matching for the interface construction. Substrate slab cell is used as a base and the film slab is wrapped into it with optionally scaling atomic coordinates (preserve coordinates in crystal/alat units).

In [ ]:
from mat3ra.made.tools.build.interface import InterfaceConfiguration, InterfaceConfigurationStrainMatcher

interface_configuration = InterfaceConfiguration(substrate_slab_configuration, film_slab_configuration, termination_pair, distance_z=3.0, vacuum=5.0, shift_a=0.0, shift_b=0.5)

interface = interface_configuration.get_material()

if ENABLE_FILM_SCALING:
    film_scaling_strain_matcher = InterfaceConfigurationStrainMatcher(interface_configuration, strain_matching_parameters=None)
    interface = film_scaling_strain_matcher.get_interfaces()[0]

### 6.2. Visualize the interface


In [ ]:
visualize([interface], repetitions=[3, 3, 1], rotation="0x")
visualize([interface], repetitions=[3, 3, 1], rotation="-90x")

### 6.3. Pass data to the outside runtime
Enrich the interface names with the strain values and pass them to the application runtime.

In [ ]:
from utils.jupyterlite import set_data

if "Interface" not in interface["name"]:
    interface["name"] = f'{interface["name"]}, Interface'

set_data("materials", [interface])